In [93]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import requests
import QuantLib as ql
import numpy as np
import json
from tradingview_screener import Query, col
import rookiepy
from sklearn.preprocessing import MinMaxScaler
from gbm_optimizer import optimize_gbm, gbm


with open("config.json", "r") as config_file:
    config = json.load(config_file)

api_key = config.get("api_key")
secret_key = config.get("secret_key")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

our_picks = ["UPST", "HOOD", "CHWY", "IBIT", "ASO", "DKNG", "PENN", "NDAQ"]



In [94]:
NASDAQ = pd.read_csv('Indexes/NASDAQ.csv')
DOWJ = pd.read_csv('Indexes/DOWJ.csv')
SP = pd.read_csv('Indexes/S&P500.csv')

def clean_data(df):
    df = df[['Company', 'Symbol']]
    df = pd.DataFrame(df).dropna()
    return df

NASDAQ = clean_data(NASDAQ)
DOWJ = clean_data(DOWJ)
SP = clean_data(SP)


In [95]:
    
def screen_stocks():
    # Get cookies for TradingView session
    cookies = rookiepy.to_cookiejar(rookiepy.chrome(['.tradingview.com']))
    
    _, df = Query().select('close','change', 'Perf.3M').where(
        col('close').between(20, 55),
        col('change').between(-4,-2),
        col('Perf.3M') > 0,
        col('exchange').isin(['AMEX', 'CBOE', 'NASDAQ', 'NYSE']),

        ).limit(1000).get_scanner_data(cookies=cookies)
    
    df[['exchange', 'ticker']] = df['ticker'].str.split(':', expand=True)
    
    return df


def get_rolling_price_change_avg(ticker: str, days: int):
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days+10)
        
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            return None, None
        
        data = data.sort_index()
        current_price = get_current_stock_price(ticker)

        data['Price_Change'] = ((current_price - data['Close'].shift(1)) / data['Close'].shift(1)) * 100

        rolling_avg = data['Price_Change'].rolling(window=min(days, len(data))).mean().iloc[-1]

        return rolling_avg
    
    except Exception as e:
        print(f"Error occurred for ticker {ticker}: {e}")
        return None, None


def get_open_interest(api_key: str, secret_key: str, option_symbol: str):
    """Fetches open interest for a given option contract using Alpaca's Trading API."""
    url = f"https://paper-api.alpaca.markets/v2/options/contracts/{option_symbol}"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        return data.get("open_interest")  # Extract OI if available
    except requests.exceptions.RequestException as e:
        print(f"Error fetching open interest for {option_symbol}: {e}")
        return None
    

def get_current_stock_price(symbol: str):

    url = "https://data.alpaca.markets/v2/stocks/trades/latest"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    params = {
        "symbols": symbol,  
        "feed": "iex" 
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  

        data = response.json()
        return data.get("trades", {}).get(symbol, {}).get("p") 

    except requests.exceptions.RequestException as e:
        print(f"Error fetching stock price: {e}")


def get_option_chain(api_key: str, secret_key: str, ticker: str, expiration_date: datetime):
    expiration_str = expiration_date.strftime("%Y-%m-%d")  
    
    url = f"https://data.alpaca.markets/v1beta1/options/snapshots/{ticker}?feed=indicative&limit=100&expiration_date={expiration_str}"
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": secret_key,
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        option_chain = data.get("snapshots", {})

        if not option_chain:
            return None

        parsed_data = []
        for symbol, details in option_chain.items():
            expiration_start = len(symbol) - 15
            option_type = "Call" if symbol[expiration_start+6] == "C" else "Put"
            strike_price = int(symbol[expiration_start+7:]) / 1000  

            open_interest = get_open_interest(api_key, secret_key, symbol)
            open_interest = int(open_interest) if open_interest is not None else 0

            greeks = details.get("greeks", {}) or {}
            latest_quote = details.get("latestQuote", {})

            print(f"\n{symbol} with oi: {open_interest}")
            parsed_data.append({
                "symbol": ticker,
                "expiration_date": expiration_str,  
                "option_type": option_type,
                "strike_price": strike_price,
                "delta": greeks.get("delta"),
                "gamma": greeks.get("gamma"),
                "rho": greeks.get("rho"),
                "theta": greeks.get("theta"),
                "vega": greeks.get("vega"),
                "implied_volatility": details.get("impliedVolatility"),
                "ask_price": latest_quote.get("ap"),
                "ask_size": latest_quote.get("as"),
                "bid_price": latest_quote.get("bp"),
                "bid_size": latest_quote.get("bs"),
                "open_interest": open_interest 

            })

        return pd.DataFrame(parsed_data)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching option chain: {e}")
        return None
    


def compute_score(contracts):
    """Compute the weighted score for contracts using normalized values."""
    temp_contracts = contracts.copy()
    
    scaler = MinMaxScaler()
    temp_contracts[['profitability_likelihood', 'return_percent', 'sortino_ratio']] = scaler.fit_transform(
        temp_contracts[['profitability_likelihood', 'return_percent', 'sortino_ratio']]
    )
    
    temp_contracts['score'] = (
        0.60 * temp_contracts['profitability_likelihood'] +
        0.40 * temp_contracts['return_percent'] 
        # 0.15 * temp_contracts['sortino_ratio']
    )
    
    contracts['score'] = temp_contracts['score']
    
    return contracts

def select_optimal_contract(contracts):
    """Select the contract with the highest weighted score while keeping original values."""
    contracts = compute_score(contracts)
    contracts = contracts.sort_values(by='score', ascending=False)
    return contracts

def compute_score(contracts):
    """Compute the weighted score for contracts using normalized values."""
    temp_contracts = contracts.copy()
    
    scaler = MinMaxScaler()
    # Note: use the new sortino_ratio in place of the sharpe_ratio
    temp_contracts[['profitability_likelihood', 'return_percent', 'sortino_ratio']] = scaler.fit_transform(
        temp_contracts[['profitability_likelihood', 'return_percent', 'sortino_ratio']]
    )
    
    temp_contracts['score'] = (
        0.60 * temp_contracts['profitability_likelihood'] +
        0.25 * temp_contracts['return_percent'] +
        0.15 * temp_contracts['sortino_ratio']
    )
    
    contracts['score'] = temp_contracts['score']
    return contracts

def select_optimal_contract(contracts):
    """Select the contract with the highest weighted score while keeping original values."""
    contracts = compute_score(contracts)
    contracts = contracts.sort_values(by='score', ascending=False)
    return contracts

def audit_contracts(expiration_date: datetime):
    # t-bill 3-month rate: 4.19% and inflation: 2.9%, scaled to daily compounding
    daily_risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/252) - 1) * 100

    #function hyperparamters
    simulation_attempts = 500
    optimizer_training_period = "1y"
    bin_length = 18
    days_to_expiration = np.busday_count(datetime.today().date(), expiration_date.date())

    all_options = pd.DataFrame(columns=[
        'symbol', 'expiration_date', 'option_type', 'strike_price', 'delta', 
        'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 
        'ask_size', 'bid_price', 'bid_size', 'open_interest'
    ])
    
    # stock screening
    candidates = screen_stocks()['ticker'].to_list()
    candidates.extend(our_picks)    
    print("Candidates:", candidates)

    for symbol in our_picks[0:2]:
        print(f"Evaluating {symbol}")
        option_chain = get_option_chain(api_key=api_key, secret_key=secret_key, ticker=symbol, expiration_date=expiration_date)

        if option_chain is None or option_chain.empty:
            continue 
        
        put_chain = option_chain[
            (option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())
        ].sort_values(by='strike_price', ascending=True)

        price = get_current_stock_price(symbol)
        optimized_mu, optimized_sigma = optimize_gbm(symbol=symbol, training_period=optimizer_training_period, bin_length=bin_length)

        for index, contract in put_chain.iterrows():
            strike_price = contract['strike_price']
            premium_collected = (contract['bid_price'] + contract['ask_price']) / 2
            simulated_returns = []
            simulated_final_prices = []
            profitable_count = 0

            if (((contract['ask_price'] - contract['bid_price']) / contract['bid_price']) * 100 > 30) and (contract['open_interest'] > 50):
                continue  

            # run simulations
            for _ in range(simulation_attempts):
                prices = gbm(
                    s0=price, mu=optimized_mu, sigma=optimized_sigma, 
                    deltaT=days_to_expiration, 
                    dt=1
                )
                final_price = prices[-1]
                # If the option expires worthless (or at-the-money), seller keeps premium.
                if final_price >= strike_price:
                    profitable_count += 1
                    net_return = (premium_collected / strike_price) * 100
                else:
                    # if assigned the net return is: premium - (loss from assignment)
                    net_return = ((premium_collected - (strike_price - final_price)) / strike_price) * 100

                simulated_returns.append(net_return)
                simulated_final_prices.append(prices[-1])

            profitability_chance = (profitable_count / simulation_attempts) * 100
            percent_return = (premium_collected / strike_price) * 100  
            avg_return = np.mean(simulated_returns)
            avg_price = np.mean(simulated_final_prices)
            
            risk_free_return = daily_risk_free_rate * days_to_expiration
            # Downside returns: only include simulated returns that are below the target.
            downside_returns = [r for r in simulated_returns if r < risk_free_return]
            if len(downside_returns) > 1:
                downside_std = np.std(downside_returns, ddof=1)
            else:
                downside_std = 0

            # Annualize the ratio (using sqrt(252/days_to_expiration))
            if downside_std != 0:
                sortino_ratio = ((avg_return - risk_free_return) / downside_std) * np.sqrt(252 / days_to_expiration)
            else:
                sortino_ratio = 0

            # Save the computed metrics into the put_chain DataFrame
                
            put_chain.at[index, 'fill_price'] = premium_collected
            put_chain.at[index, 'current_price'] = price
            put_chain.at[index, 'final_price'] = avg_price
            put_chain.at[index, 'profitability_likelihood'] = profitability_chance
            put_chain.at[index, 'return_percent'] = percent_return
            put_chain.at[index, 'average_return'] = avg_return
            put_chain.at[index, 'sortino_ratio'] = sortino_ratio


        all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)

    return all_options


In [96]:

priced_contracts = audit_contracts(expiration_date = datetime(year=2025, month=2, day=14))

Candidates: ['BITX', 'CCL', 'ASTS', 'GH', 'NCLH', 'TOST', 'MTCH', 'BITU', 'CHWY', 'MAGS', 'NXT', 'DJT', 'FLEX', 'VFC', 'GAP', 'GTES', 'ALGM', 'ERJ', 'KGS', 'KD', 'GEO', 'YPF', 'TSEM', 'BBIO', 'BRZE', 'TWST', 'VIRT', 'AAOI', 'AESI', 'FIVN', 'VITL', 'AZTA', 'RDW', 'GDXU', 'DXYZ', 'CHEF', 'TTMI', 'APLS', 'CUK', 'GRAL', 'FHB', 'SEI', 'ARIS', 'SNN', 'MLPA', 'SNRE', 'NVCR', 'SPT', 'CENTA', 'REVG', 'UNFI', 'HRMY', 'AORT', 'BBW', 'EVER', 'CEVA', 'MAGX', 'PAHC', 'BTCL', 'AVNW', 'ETD', 'CECO', 'EFSI', 'ELMD', 'CTRI', 'CLW', 'MQQQ', 'QQQU', 'UVSP', 'RICK', 'BBSI', 'SMC', 'UFCS', 'MOFG', 'AMZP', 'IPX', 'UNTY', 'LAKE', 'WLKP', 'RCKY', 'BWFG', 'FSTR', 'TESL', 'GNTY', 'HBT', 'BTFX', 'EURL', 'VLGEA', 'PLBC', 'CCRD', 'AGFY', 'UCC', 'HNVR', 'PRNT', 'QQQP', 'IWFL', 'EVAV', 'LARK', 'QQQW', 'LUX', 'EATZ', 'FINE', 'BULD', 'MAKX', 'CXRN', 'UPST', 'HOOD', 'CHWY', 'IBIT', 'ASO', 'DKNG', 'PENN', 'NDAQ']
Evaluating UPST

UPST250214C00047500 with oi: 0

UPST250214C00056000 with oi: 11

UPST250214C00086000 with oi

[*********************100%***********************]  1 of 1 completed
/var/folders/90/x6spqd2j4cbb38yrckqjm8840000gn/T/ipykernel_26922/2248170905.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)


Evaluating HOOD

HOOD250214C00051000 with oi: 812

HOOD250214C00037000 with oi: 22

HOOD250214C00055000 with oi: 3114

HOOD250214C00031000 with oi: 7

HOOD250214C00063000 with oi: 365

HOOD250214C00035500 with oi: 11

HOOD250214P00033000 with oi: 46

HOOD250214C00054000 with oi: 1289

HOOD250214P00040500 with oi: 55

HOOD250214P00044500 with oi: 41

HOOD250214C00032000 with oi: 14

HOOD250214C00060000 with oi: 7739

HOOD250214C00073000 with oi: 17

HOOD250214C00062000 with oi: 733

HOOD250214C00053000 with oi: 1676

HOOD250214C00025000 with oi: 9

HOOD250214C00036000 with oi: 3

HOOD250214P00040000 with oi: 1679

HOOD250214C00033500 with oi: 6

HOOD250214C00040000 with oi: 130

HOOD250214C00068000 with oi: 141

HOOD250214P00033500 with oi: 8

HOOD250214P00045000 with oi: 1295

HOOD250214P00041000 with oi: 306

HOOD250214C00039500 with oi: 203

HOOD250214P00034000 with oi: 55

HOOD250214C00066000 with oi: 146

HOOD250214P00046500 with oi: 266

HOOD250214C00046500 with oi: 120

HOOD25021

[*********************100%***********************]  1 of 1 completed


In [97]:
contracts = select_optimal_contract(priced_contracts)
print('length',len(contracts))
display(contracts)

length 17


,symbol,expiration_date,option_type,strike_price,delta,gamma,rho,theta,vega,implied_volatility,ask_price,ask_size,bid_price,bid_size,open_interest,profitability_likelihood,return_percent,average_return,sortino_ratio,current_price,fill_price,final_price,score
2,UPST,2025-02-14,Put,61.0,-0.2709,0.0180,-0.0043,-0.4329,0.0317,1.9211,3.55,1,3.50,20,270,96.2,5.778689,5.676453,33.435143,68.88,3.525,70.556141,0.823743
3,UPST,2025-02-14,Put,62.0,-0.2919,0.0186,-0.0046,-0.4507,0.0329,1.9297,3.98,1,3.88,20,408,95.2,6.338710,6.161287,28.583748,68.88,3.930,70.366895,0.802910
4,UPST,2025-02-14,Put,63.0,-0.3136,0.0190,-0.0050,-0.4702,0.0340,1.9501,4.42,10,4.38,20,670,91.4,6.984127,6.665963,28.016048,68.88,4.400,70.056229,0.769281
1,UPST,2025-02-14,Put,60.0,-0.2501,0.0173,-0.0039,-0.4137,0.0305,1.9124,3.17,3,3.12,20,5200,97.6,5.241667,5.155368,12.037378,68.88,3.145,70.369420,0.732209
5,UPST,2025-02-14,Put,64.0,-0.3343,0.0196,-0.0054,-0.4808,0.0349,1.9428,4.87,5,4.73,22,772,84.8,7.500000,6.958050,33.195960,68.88,4.800,69.851379,0.724405
6,UPST,2025-02-14,Put,65.0,-0.3559,0.0199,-0.0057,-0.4961,0.0357,1.9587,5.43,10,5.18,24,480,82.0,8.161538,7.380731,29.977304,68.88,5.305,70.072309,0.691406
0,UPST,2025-02-14,Put,58.0,-0.2104,0.0158,-0.0033,-0.3736,0.0276,1.9019,2.54,29,2.42,13,947,100.0,4.275862,4.275862,0.000000,68.88,2.480,69.970134,0.684802
16,HOOD,2025-02-14,Put,47.5,-0.1273,0.0246,-0.0015,-0.1252,0.0161,1.0966,0.61,182,0.54,11,211,100.0,1.210526,1.210526,0.000000,55.92,0.575,57.113231,0.613219
15,HOOD,2025-02-14,Put,47.0,-0.1196,0.0228,-0.0014,-0.1236,0.0154,1.1307,0.56,74,0.54,105,841,99.8,1.170213,1.166802,0.000000,55.92,0.550,56.974473,0.609849
13,HOOD,2025-02-14,Put,46.0,-0.0973,0.0194,-0.0011,-0.1083,0.0133,1.1484,0.45,567,0.42,16,800,100.0,0.945652,0.945652,0.000000,55.92,0.435,57.295107,0.607034


In [98]:
# TODO
"""
- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer
- Find better metric for optimizer
- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins
"""




'\n- Figure out way to normalize stock price, whether that is min max of the range of the price(shoudl help optimizer\n- Find better metric for optimizer\n- Try binning, so like get past 10 years of AAPL, seperate into bins of 20 or n trading days, train optimzer on each one. Then the hyperparameters can be weighted to have more bias towards more recent bins\n'

In [99]:
# Archive


# def gbm(s0, mu, sigma, deltaT, dt):
#     """
#     Models a stock price S(t) using the Wiener process W(t) as
#     `S(t) = S(0).exp{(mu-(sigma^2/2).t)+sigma.W(t)}`
    
#     Arguments:
#         s0: Initial stock price, default 100
#         mu: 'Drift' of the stock (upwards or downwards), default 0.2
#         sigma: 'Volatility' of the stock, default 0.68
#         deltaT: The time period for which the future prices are computed, default 52 (as in 52 weeks)
#         dt: The granularity of the time-period, default 0.1
    
#     Returns:
#         time_vector: array of time steps
#         s: array with the simulated stock prices over the time-period deltaT
#     """
#     n_step = int(deltaT / dt)  # Number of time steps
#     time_vector = np.linspace(0, deltaT, num=n_step)  # Time vector
    
#     # Wiener process: cumulative sum of random normal increments
#     random_increments = np.random.normal(0, 1, size=n_step) * np.sqrt(dt)
#     weiner_process = np.cumsum(random_increments)
    
#     # Stock price simulation
#     stock_var = (mu - (sigma**2 / 2)) * time_vector
#     s = s0 * np.exp(stock_var + sigma * weiner_process)
    
#     return s



# def aobjective(params, real_prices, s0):
#     """Objective function for optimization."""
#     mu, sigma = params  # Unpack parameters
#     gbm_prices = gbm(s0, mu, sigma, deltaT=len(real_prices), dt=1)
#     return mean_squared_error(real_prices, gbm_prices)

# def aoptimize_gbm(symbol: str, training_period: str, bin_length: int):
#     """
#     Optimize μ and σ over multiple time bins, weighting recent periods more.
#     """
#     # Fetch real stock data (past 5 years)
#     stock_data = yf.download(symbol, period=training_period, interval="1d")
#     real_prices = stock_data["Close"].dropna().values

#     num_bins = len(real_prices) // bin_length
#     weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

#     mu_values, sigma_values, mses = [], [], []

#     for i in range(num_bins):
#         bin_prices = real_prices[i * bin_length : (i + 1) * bin_length]
#         s0 = bin_prices[0]

#         bounds = [(-0.3, 0.3), (0.001, 0.35)]

#         result = differential_evolution(objective, bounds, args=(bin_prices, s0))
#         best_mu, best_sigma = result.x
#         best_mse = result.fun

#         mu_values.append(best_mu)
#         sigma_values.append(best_sigma)
#         mses.append(best_mse)

#     weight_sum = np.sum(weights)
#     avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
#     avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

#     print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

#     return avg_mu, avg_sigma



    # def filter_stocks(rolling_change_period): 
    # filtered_stocks = set()
    # stocks = screen_stocks()

    # for index, stock in stocks.iterrows():
    #     try:
    #         today_change, rolling_avg = get_rolling_price_change_avg(stock['ticker'], days=rolling_change_period)
    #         current_price = get_current_stock_price(stock['ticker'])

    #         # Skip if any value is None
    #         if None in (today_change, rolling_avg, current_price):
    #             print(f"Skipping {stock['ticker']} due to missing data.")
    #             continue

    #         # Apply filtering conditions
    #         if (rolling_avg > 0.00): 
    #             filtered_stocks.add(stock['ticker'])

    #     except Exception as e:
    #         print(f"Skipping {stock['ticker']} due to error: {e}")
    #         continue
    
    # return filtered_stocks

    #def ORIGINAL_LOGIC_FOR _AUDITING_OPTIONS()
    # simulation_attempts = 200
    # optimizer_training_period = "2y"
    # bin_length = 20
    # rolling_change_period = 15
    # expiration_date = datetime(year=2025, month=2, day=14) 
    # all_options = pd.DataFrame(columns=['symbol', 'expiration_date', 'option_type', 'strike_price', 'delta', 'gamma', 'rho', 'theta', 'vega', 'implied_volatility', 'ask_price', 'ask_size', 'bid_price', 'bid_size'])
    # candidates = ["AAPL", "AMD"]
    # # filter_stocks(rolling_change_period=rolling_change_period)

    # # t-bill 3-month rate: 4.19%, inflation rate: 2.9% -> scaled to weekly
    # risk_free_rate = (((1 + 0.0419) / (1 + 0.029)) ** (1/52) - 1) * 100

    # print(candidates)

    # for symbol in candidates:
    #     option_chain = get_option_chain(api_key=api_key, secret_key=secret_key, ticker=symbol, expiration_date=expiration_date)
    #     put_chain = option_chain[(option_chain['option_type'] == 'Put') & (option_chain['rho'].notna())].sort_values(by='strike_price', ascending=True)

    #     if option_chain is None or option_chain.empty:
    #         continue 

    #     price = get_current_stock_price(symbol)
    #     optimized_mu, optimized_sigma = optimize_gbm(symbol=symbol, training_period=optimizer_training_period, bin_length=bin_length)

    #     profitability_chances = []
    #     percent_returns = []

    #     for index, contract in put_chain.iterrows():
    #         count = 0
    #         strike_price = contract['strike_price']

    #         for i in range(simulation_attempts):
    #             prices = gbm(s0=price, mu=optimized_mu, sigma=optimized_sigma, 
    #                 deltaT=np.busday_count(datetime.today().date(), datetime.strptime(contract['expiration_date'], "%Y-%m-%d").date()), dt=1)  
    #             if prices[-1] > strike_price:
    #                 count += 1
    #         profitability_chance = (count / simulation_attempts) * 100
    #         profit = (contract['bid_price']*contract['bid_size'] + contract['ask_price']*contract['ask_size']) / (contract['ask_size'] + contract['bid_size'])
    #         percent_return = (profit / (strike_price)) * 100

    #         profitability_chances.append(profitability_chance)
    #         percent_returns.append(percent_return)
    #     put_chain['profitability_percent'] = profitability_chances
    #     put_chain['percent_return'] = percent_returns
    #     put_chain['expected_value'] = put_chain['profitability_percent'] * put_chain['percent_return']
    #     put_chain['current_price'] = price
    #     if put_chain['percent_return'].std() != 0:
    #         put_chain['sortino_ratio'] = (put_chain['percent_return'] - risk_free_rate) / put_chain['percent_return'].std()
    #     else:
    #         put_chain['sortino_ratio'] = 0  # Avoid division by zero
    #     all_options = pd.concat([all_options, put_chain], ignore_index=True, copy=False)


# def gbm_vs_real_graph(symbol, mu, sigma, period):
#     stock_data = yf.download(symbol, period=period, interval="1d")
#     real_prices = stock_data["Close"].dropna().values
#     time_steps = np.arange(len(real_prices))


#     gbm_path = gbm(s0 = real_prices[0], mu=mu, sigma=sigma, deltaT=len(real_prices), dt=1)
#     plt.figure(figsize=(10, 5))
#     plt.plot(time_steps, real_prices, label="Real Prices", color="blue")
#     plt.plot(time_steps, gbm_path, label="GBM Simulated", linestyle="dashed", color="red")
    
#     plt.xlabel("Time (Days)")
#     plt.ylabel("Price")
#     plt.title(f"GBM vs Real Prices for {symbol}")
#     plt.legend()
#     plt.grid()
#     plt.show()

# def multithread_optimize_bin(bin_prices, bin_size, weights, i):
#     s0 = bin_prices[0]

#     # Define the bounds for optimization
#     bounds = [(-0.3, 0.3), (0.001, 0.30)]

#     # Run the optimizer for the bin
#     result = differential_evolution(objective, bounds, args=(bin_prices, s0))
#     best_mu, best_sigma = result.x
#     best_mse = result.fun

#     print(f"Bin {i+1}: μ = {best_mu:.4f}, σ = {best_sigma:.4f}, MSE = {best_mse:.4f}")
#     return best_mu, best_sigma, best_mse

# def multithread_optimize_gbm(symbol): 
    # """
    # Optimize μ and σ over multiple time bins, weighting recent periods more.
    # """
    # # Fetch real stock data (past 2 years)
    # stock_data = yf.download(symbol, period="2y", interval="1d")
    # real_prices = stock_data["Close"].dropna().values

    # # Split into bins of 20 trading days
    # bin_size = 20
    # num_bins = len(real_prices) // bin_size
    # weights = np.linspace(1, 2, num_bins)  # Increasing weights for recent bins

    # # Initialize containers for results
    # mu_values, sigma_values, mses = [], [], []

    # # Use concurrent.futures for parallel processing of bins
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #     futures = []
    #     for i in range(num_bins):
    #         bin_prices = real_prices[i * bin_size : (i + 1) * bin_size]
    #         futures.append(executor.submit(optimize_bin, bin_prices, bin_size, weights, i))
        
    #     for future in concurrent.futures.as_completed(futures):
    #         best_mu, best_sigma, best_mse = future.result()
    #         mu_values.append(best_mu)
    #         sigma_values.append(best_sigma)
    #         mses.append(best_mse)

    # # Compute weighted averages
    # weight_sum = np.sum(weights)
    # avg_mu = np.sum(np.array(mu_values) * weights) / weight_sum
    # avg_sigma = np.sum(np.array(sigma_values) * weights) / weight_sum

    # print(f"\nFinal Weighted Averages: μ = {avg_mu:.4f}, σ = {avg_sigma:.4f}")

    # return avg_mu, avg_sigma